## Practice with pandas

The pandas.DataFrame type lets you join pandas.Series type columns into a multi-column data table, complete with row and column names of your choice, both re-orderable.

Once you have a DataFrame defined, adding new columns based on the old, getting summary statistics, applying functions, generating visualizations, is all within reach.

In [1]:
# be skeptical of code like this -- but maybe?
name = 'math'
exec(f'import {name} as clc')

In [2]:
clc.sin(clc.radians(90)) # remembering trig

1.0

In [3]:
# and now for something completely different...
import numpy as np
import pandas as pd

Lets stack up a column of polyhedron names, using a kind of jargon or shorthand.

In [79]:
shapes = np.array(["Tetra", "Cubocta", "Icosa", "Cube", "Octa", 
                    "RT5", "RT5+", "RD", "Icosa", "Cubocta", 
                    "SuperRT", "Cube"], 
                    dtype=np.str_) 
shapes

array(['Tetra', 'Cubocta', 'Icosa', 'Cube', 'Octa', 'RT5', 'RT5+', 'RD',
       'Icosa', 'Cubocta', 'SuperRT', 'Cube'], dtype='<U7')

So far that's a `numpy.ndarray` we've created.  Now lets bring that into a Series.

In [80]:
shapes_col = pd.Series(shapes, name="Shape")
shapes_col

0       Tetra
1     Cubocta
2       Icosa
3        Cube
4        Octa
5         RT5
6        RT5+
7          RD
8       Icosa
9     Cubocta
10    SuperRT
11       Cube
Name: Shape, dtype: object

The vertical Series, a column of some data type (dtype), is the building block of the DataFrame, which sets them side by side in a tabular arrangement.

In [81]:
# geometric constants
phi = (1 + clc.sqrt(5))/2

# volumes of specific tetrahedral wedges
Emod  = clc.sqrt(2)/8 * 1/phi**3
Emod3 = clc.sqrt(2)/8
Smod  = (phi**-5) / 2

Sfactor = Smod/Emod

S3 = clc.sqrt(9/8)

# defined to have edges = 2R or 1D
Icosa = 100 * E3 + 20 * E

# volumes corresponding to our shapes
volumes = np.array([1, 2.5, 2.5 * Sfactor**2, 3, 4, 5, 120 * E, 6, Icosa, 20, 20 * S3, 24],  dtype=np.float)

In [82]:
volumes_col = pd.Series(volumes, name="IVM Volume")  # turn np.array into a pd.Series

In [83]:
volumes_col

0      1.000000
1      2.500000
2      2.917961
3      3.000000
4      4.000000
5      5.000000
6      5.007758
7      6.000000
8     18.512296
9     20.000000
10    21.213203
11    24.000000
Name: IVM Volume, dtype: float64

In [84]:
vols_table = pd.DataFrame(volumes_col)
vols_table.index = shapes_col  #  the shapes column is the index

In [85]:
vols_table

,IVM Volume
Shape,
Tetra,1.000000
Cubocta,2.500000
Icosa,2.917961
Cube,3.000000
Octa,4.000000
RT5,5.000000
RT5+,5.007758
RD,6.000000
Icosa,18.512296


In [86]:
vols_table['XYZ Volume'] = vols_table['IVM Volume'] * 1/S3

In [87]:
vols_table

,IVM Volume,XYZ Volume
Shape,,
Tetra,1.000000,0.942809
Cubocta,2.500000,2.357023
Icosa,2.917961,2.751080
Cube,3.000000,2.828427
Octa,4.000000,3.771236
RT5,5.000000,4.714045
RT5+,5.007758,4.721360
RD,6.000000,5.656854
Icosa,18.512296,17.453560


Practice with `df.loc[rows, columns]`.

In [89]:
vols_table.iloc[11]  # entire row

IVM Volume    24.000000
XYZ Volume    22.627417
Name: Cube, dtype: float64

In [90]:
vols_table.iloc[0]  # entire row

IVM Volume    1.000000
XYZ Volume    0.942809
Name: Tetra, dtype: float64

In [91]:
vols_table.loc['SuperRT', 'XYZ Volume'] # specific cell

20.0

Now lets add some constituent modules that may be used to assemble the above shapes.

In [64]:
modules = np.array(["A","B", "T", "E", "S"],
                    dtype=np.str_) 

mods_col = pd.Series(modules, name="Shape")

mod_vols = np.array([1/24, 1/24, 1/24, E, (phi**-5) / 2],  dtype=np.float)
mod_vols_col = pd.Series(mod_vols, name="IVM Volume")

mods_table = pd.DataFrame(mod_vols_col)
mods_table.index = mods_col

In [17]:
mods_table

,IVM Volume
Shape,
A,0.041667
B,0.041667
T,0.041667
E,0.041731
S,0.045085


In [66]:
mods_table['XYZ Volume'] = mods_table['IVM Volume'] * 1/S3

In [19]:
mods_table

,IVM Volume,XYZ Volume
Shape,,
A,0.041667,0.039284
B,0.041667,0.039284
T,0.041667,0.039284
E,0.041731,0.039345
S,0.045085,0.042507


And now it's time to assemble the full table.

In [92]:
pd.concat([mods_table, vols_table])

,IVM Volume,XYZ Volume
Shape,,
A,0.041667,0.039284
B,0.041667,0.039284
T,0.041667,0.039284
E,0.041731,0.039345
S,0.045085,0.042507
Tetra,1.000000,0.942809
Cubocta,2.500000,2.357023
Icosa,2.917961,2.751080
Cube,3.000000,2.828427


In [94]:
CH = pd.concat([mods_table, vols_table])

In [101]:
CH.info()

<class 'pandas.core.frame.DataFrame'>
Index: 17 entries, A to Cube
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   IVM Volume  17 non-null     float64
 1   XYZ Volume  17 non-null     float64
 2   Comments    15 non-null     object 
dtypes: float64(2), object(1)
memory usage: 1.1+ KB


In [97]:
# df['new'] = pd.Series(dtype='int')
CH['Comments'] = pd.Series(dtype='str_')

In [114]:
CH.iloc[0, -1] = '24 make a Tetra'
CH.iloc[1, -1] = 'AAB = BAA = Mite'
CH.iloc[2, -1] = '1/120 RT5'
CH.iloc[3, -1] = '1/120 RT5+'
CH.iloc[4, -1] = '(φ**-5) / 2'
CH.loc["Tetra", "Comments"] = "edges D, from 4 IVM balls"
CH.iloc[6, -1] = 'some faces flush with Octa 4'
CH.iloc[7, -1] = 'some faces flush with Octa 4'
CH.iloc[8, -1] = 'Duo-Tet, face diagonals = D'
CH.iloc[9, -1] = 'Dual of Cube, edges D'
CH.iloc[10, -1] = '120 T mods'
CH.iloc[11, -1] = '120 E mods'
CH.loc['RD', 'Comments'] = 'long diagonals = D, sphere domain'
CH.iloc[13, -1] = 'edges = D'
CH.iloc[14, -1] = 'edges = D, 1F, 12-balls around nuclear ball'
CH.iloc[15, -1] = 'icosa of edges D + dual'
CH.iloc[16, -1] = 'face diagonals = 2D, 2F'

In [112]:
CH

,IVM Volume,XYZ Volume,Comments
Shape,,,
A,0.041667,0.039284,24 make a Tetra
B,0.041667,0.039284,AAB = BAA = Mite
T,0.041667,0.039284,1/120 RT5
E,0.041731,0.039345,1/120 RT5+
S,0.045085,0.042507,(φ**-5) / 2
Tetra,1.000000,0.942809,"edges D, from 4 IVM balls"
Cubocta,2.500000,2.357023,some faces flush with Octa 4
Icosa,2.917961,2.751080,some faces flush with Octa 4
Cube,3.000000,2.828427,"Duo-Tet, face diagonals = D"


In [113]:
CH

,IVM Volume,XYZ Volume,Comments
Shape,,,
A,0.041667,0.039284,24 make a Tetra
B,0.041667,0.039284,AAB = BAA = Mite
T,0.041667,0.039284,1/120 RT5
E,0.041731,0.039345,1/120 RT5+
S,0.045085,0.042507,(φ**-5) / 2
Tetra,1.000000,0.942809,"edges D, from 4 IVM balls"
Cubocta,2.500000,2.357023,some faces flush with Octa 4
Icosa,2.917961,2.751080,some faces flush with Octa 4
Cube,3.000000,2.828427,"Duo-Tet, face diagonals = D"
